In [5]:
from radd.tools import vis
from radd import build
import radd
# read in example dataframe 
data = radd.load_example_data()
dpm = build.Model(kind='xdpm', data=data, depends_on={'v':'Cond'}, fit_on='average')

In [9]:
def animate_dpm(model):
    """ to render animation within a notebook :
        vis.render_animation(vis.animated_dpm_example(MODEL))
    """
    from radd.tools import theta
    from matplotlib import animation
    from copy import deepcopy
    params = deepcopy(model.inits)
    bound=theta.scalarize_params(params)['a']
    x, gtraces, straces, xi, yi, nframes = vis.gen_re_traces(model, params)
    f, axes = build_decision_axis(onset=x[0][0], bound=bound)
    glines = [axes[i].plot([], [], linewidth=1.5)[0] for i, n in enumerate(gtraces)]
    slines = [axes[i].plot([xi[i]], [yi[i]], linewidth=1.5)[0] for i, n in enumerate(straces)]
    f_args = (x, gtraces, glines, straces, slines, params, xi, yi)
    anim=animation.FuncAnimation(f, vis.re_animate_multiax, fargs=f_args, frames=nframes, interval=4, blit=True)
    return anim

def build_decision_axis(onset, bound, ssd=np.array([300, 400]), tb=650):
    # init figure, axes, properties
    import matplotlib.pyplot as plt
    f, axes = plt.subplots(len(ssd), 1, figsize=(7, 7), dpi=300)
    #f.subplots_adjust(wspace=.05, top=.1, bottom=.1)
    f.subplots_adjust(hspace=.05, top=.99, bottom=.09)
    w = tb + 40
    h = bound
    start = onset - 80
    # c=["#e74c3c", '#27ae60', '#4168B7', '#8E44AD']
    for i, ax in enumerate(axes):
        plt.setp(ax, xlim=(start - 1, w + 1), ylim=(0 - (.01 * h), h + (.01 * h)))
        ax.vlines(x=ssd[i], ymin=0, ymax=h, color="#e74c3c", lw=2.5, alpha=.5)
        ax.hlines(y=h, xmin=start, xmax=w, color='k')
        ax.hlines(y=0, xmin=start, xmax=w, color='k')
        ax.vlines(x=tb, ymin=0, ymax=h, color='#2043B0', lw=2.5, linestyle='-', alpha=.5)
        ax.vlines(x=start + 2, ymin=0, ymax=h, color='k')
        ax.text(ssd[i] + 10, h * .88, str(ssd[i]) + 'ms', fontsize=19)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_xticks([])
        ax.set_yticks([])
    sns.despine(top=True, right=True, bottom=True, left=True)
    return f, axes

def render_animation(anim):
    import matplotlib.pyplot as plt
    from IPython.display import HTML
    plt.close(anim._fig)
    return HTML(vis.anim_to_html(anim))

In [1]:
movpath = os.path.join(_dbpath, 'anim.mp4')
anim = animate_dpm(dpm)
render_animation(anim)

NameError: name 'dpm' is not defined

In [ ]:
anim.save(movpath, dpi=200, extra_args=['-vcodec', 'libx264', '-pix_fmt', 'yuv420p'])
video = open(movpath, "rb").read()
anim._encoded_video = video.encode("base64")
render_animation(anim)